In [1]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
n_samples = 20000
sample_inputs = []
sample_outputs = []
n_tokens = 61

patterns = [
    lambda i: list(range(i, i+3)),
    lambda i: list(range(i, i-3, -1))
]

for n in range(n_samples):
    sample_in = []
    sample_out = []
    for line_number in range(3):
        i = np.random.randint(10,50)
        #line = list(range(i, i+3))
        pattern = np.random.randint(len(patterns))
        line = patterns[pattern](i)
        '''if line_number == 0:
            sample_in.append([line[0]] + line)
        else:
            sample_in.append([0] + line)'''
            
        sample_in.append([line[0]] + line)
        sample_out.append(line + [60])
        
        
        #sample.append(np.arange(i, i+3))
    #samples.append(sample)
    sample_inputs.append(sample_in)
    sample_outputs.append(sample_out)
    
X = np_utils.to_categorical(sample_inputs, num_classes=n_tokens)
Y = np_utils.to_categorical(sample_outputs, num_classes=n_tokens)

In [3]:
sample_inputs[:10]

[[[16, 16, 15, 14], [20, 20, 19, 18], [37, 37, 36, 35]],
 [[28, 28, 27, 26], [36, 36, 37, 38], [24, 24, 23, 22]],
 [[19, 19, 20, 21], [34, 34, 35, 36], [18, 18, 19, 20]],
 [[39, 39, 40, 41], [24, 24, 23, 22], [30, 30, 31, 32]],
 [[16, 16, 15, 14], [49, 49, 50, 51], [44, 44, 43, 42]],
 [[29, 29, 28, 27], [38, 38, 39, 40], [34, 34, 33, 32]],
 [[39, 39, 38, 37], [15, 15, 14, 13], [31, 31, 32, 33]],
 [[38, 38, 39, 40], [12, 12, 11, 10], [24, 24, 25, 26]],
 [[27, 27, 26, 25], [44, 44, 43, 42], [30, 30, 29, 28]],
 [[35, 35, 36, 37], [26, 26, 25, 24], [44, 44, 43, 42]]]

In [4]:
latent_dim = 512

# Keras Model

inputs = [Input(shape=(None, n_tokens)) for i in range(3)]
lstms = [LSTM(latent_dim, return_state=True, return_sequences=True) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[lstm_h[i-1], lstm_c[i-1]])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i])
        
denses = [Dense(n_tokens, activation='softmax') for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]

model = Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_1 (InputLayer)                                              (None, None, 61)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
lstm_1 (LSTM)                                                     [(None, None, 512), (None, 512), (None, 512 1175552                 input_1[0][0]                                                 

In [5]:
model.fit([X[:,0,:], X[:,1,:], X[:,2,:]], [Y[:,0,:], Y[:,1,:], Y[:,2,:]], batch_size=64, epochs=20)

Epoch 1/20
20000/20000 [==============================] - 13s 641us/step - loss: 5.4413 - dense_1_loss: 2.0439 - dense_2_loss: 1.7076 - dense_3_loss: 1.6898
Epoch 2/20
20000/20000 [==============================] - 10s 520us/step - loss: 1.1170 - dense_1_loss: 0.5336 - dense_2_loss: 0.3160 - dense_3_loss: 0.2674
Epoch 3/20
20000/20000 [==============================] - 10s 514us/step - loss: 0.6773 - dense_1_loss: 0.2698 - dense_2_loss: 0.2066 - dense_3_loss: 0.2009
Epoch 4/20
20000/20000 [==============================] - 10s 517us/step - loss: 0.6070 - dense_1_loss: 0.2238 - dense_2_loss: 0.1936 - dense_3_loss: 0.1896
Epoch 5/20
20000/20000 [==============================] - 10s 513us/step - loss: 0.5838 - dense_1_loss: 0.2119 - dense_2_loss: 0.1879 - dense_3_loss: 0.1840
Epoch 6/20
20000/20000 [==============================] - 10s 513us/step - loss: 0.5685 - dense_1_loss: 0.2051 - dense_2_loss: 0.1839 - dense_3_loss: 0.1796
Epoch 7/20
20000/20000 [==============================] - 

In [7]:
model.fit([X[:,0,:], X[:,1,:], X[:,2,:]], [Y[:,0,:], Y[:,1,:], Y[:,2,:]], batch_size=128, epochs=10)

Epoch 1/10
20000/20000 [==============================] - 6s 283us/step - loss: 0.3701 - dense_1_loss: 0.1784 - dense_2_loss: 0.1651 - dense_3_loss: 0.0266
Epoch 2/10
20000/20000 [==============================] - 6s 282us/step - loss: 0.3663 - dense_1_loss: 0.1783 - dense_2_loss: 0.1652 - dense_3_loss: 0.0229
Epoch 3/10
20000/20000 [==============================] - 6s 276us/step - loss: 0.3628 - dense_1_loss: 0.1781 - dense_2_loss: 0.1648 - dense_3_loss: 0.0198
Epoch 4/10
20000/20000 [==============================] - 5s 271us/step - loss: 0.3612 - dense_1_loss: 0.1777 - dense_2_loss: 0.1648 - dense_3_loss: 0.0188
Epoch 5/10
20000/20000 [==============================] - 6s 287us/step - loss: 0.3597 - dense_1_loss: 0.1779 - dense_2_loss: 0.1644 - dense_3_loss: 0.0174
Epoch 6/10
20000/20000 [==============================] - 6s 281us/step - loss: 0.3582 - dense_1_loss: 0.1778 - dense_2_loss: 0.1642 - dense_3_loss: 0.0162
Epoch 7/10
20000/20000 [==============================] - 6s 282

In [8]:
model.fit([X[:,0,:], X[:,1,:], X[:,2,:]], [Y[:,0,:], Y[:,1,:], Y[:,2,:]], batch_size=128, epochs=10)

Epoch 1/10
20000/20000 [==============================] - 6s 281us/step - loss: 0.3536 - dense_1_loss: 0.1770 - dense_2_loss: 0.1632 - dense_3_loss: 0.0134
Epoch 2/10
20000/20000 [==============================] - 6s 304us/step - loss: 0.3524 - dense_1_loss: 0.1769 - dense_2_loss: 0.1633 - dense_3_loss: 0.0123
Epoch 3/10
20000/20000 [==============================] - 6s 283us/step - loss: 0.3527 - dense_1_loss: 0.1767 - dense_2_loss: 0.1630 - dense_3_loss: 0.0130
Epoch 4/10
20000/20000 [==============================] - 6s 283us/step - loss: 0.3516 - dense_1_loss: 0.1766 - dense_2_loss: 0.1629 - dense_3_loss: 0.0120
Epoch 5/10
20000/20000 [==============================] - 6s 280us/step - loss: 0.3513 - dense_1_loss: 0.1765 - dense_2_loss: 0.1628 - dense_3_loss: 0.0120
Epoch 6/10
20000/20000 [==============================] - 6s 284us/step - loss: 0.3513 - dense_1_loss: 0.1766 - dense_2_loss: 0.1627 - dense_3_loss: 0.0120
Epoch 7/10
20000/20000 [==============================] - 6s 287

In [9]:
generator_inputs = [Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens))]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True)]
generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
generator_lstm_out[0], generator_lstm_h[0], generator_lstm_c[0] = generator_lstms[0](
    generator_inputs[0]
)
generator_denses = [Dense(n_tokens, activation='softmax')]
generator_outputs = [generator_denses[0](generator_lstm_out[0])]

In [10]:
generator_model_line0 = Model(generator_inputs[0], [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])
generator_model_line0.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (1, None, 61)             0         
_________________________________________________________________
lstm_4 (LSTM)                [(1, None, 512), (1, 512) 1175552   
_________________________________________________________________
dense_4 (Dense)              (1, None, 61)             31293     
Total params: 1,206,845
Trainable params: 1,206,845
Non-trainable params: 0
_________________________________________________________________


In [11]:
generator_lstms[0].set_weights(lstms[0].get_weights())
generator_denses[0].set_weights(denses[0].get_weights())

In [23]:
generator_model_line0.reset_states()
line = [44]

def f():
    while line[-1] != 60:
        yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
while line[-1] != 60:
    character, h, c = generator_model_line0.predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[44, 44, 43, 42, 60]

In [13]:
generator_inputs_h = [None, Input(shape=(latent_dim,))]
generator_inputs_c = [None, Input(shape=(latent_dim,))]

generator_inputs.append(Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)))
generator_lstms.append(LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True))

generator_lstm_out[1], generator_lstm_h[1], generator_lstm_c[1] = generator_lstms[1](
    generator_inputs[1], initial_state=[generator_inputs_h[1], generator_inputs_c[1]]
)
generator_denses.append(Dense(n_tokens, activation='softmax'))
generator_outputs.append(generator_denses[1](generator_lstm_out[1]))

generator_model_line1 = Model(
    [generator_inputs[1], generator_inputs_h[1], generator_inputs_c[1]],
    [generator_outputs[1], generator_lstm_h[1], generator_lstm_c[1]]
)
generator_model_line1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (1, None, 61)        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(1, None, 512), (1, 1175552     input_7[0][0]                    
                                                                 input_5[0][0]                    
          

In [14]:
generator_lstms[1].set_weights(lstms[1].get_weights())
generator_denses[1].set_weights(denses[1].get_weights())

In [24]:
#generator_model_line1.reset_states()
#generator_line1_lstm.reset_states()
#generator_lstms[1].reset_states()
line = [20]
#h = np.array([np.zeros(256)])
#c = np.array([np.zeros(256)])

def f():
    while line[-1] != 60:
        yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
        
while line[-1] != 60:
    character, h, c = generator_model_line1.predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[20, 20, 19, 18, 60]